In [3]:
import os
import glob

# Path to your dataset folder
# Each class should be in a separate folder inside this directory
dataset_path = "dataset"

# List all subfolders (classes)
classes = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

print("Found classes:", classes)
print("Total classes:", len(classes))
# Count images in each class
for cls in classes:
    class_path = os.path.join(dataset_path, cls)
    # Assuming common image extensions
    image_files = glob.glob(os.path.join(class_path, "*.[pjPJ]*[ngNG]*"))  # jpg, jpeg, png
    print(f"Class '{cls}' has {len(image_files)} images")


Found classes: ['Acne And Rosacea Photos', 'Malignant', 'Fu Athlete Foot', 'Eczema Photos', 'Fu Ringworm', 'Benign', 'Exanthems And Drug Eruptions', 'Lupus And Other Connective Tissue Diseases', 'Ba Impetigo', 'Bullous Disease Photos', 'Nail Fungus And Other Nail Disease', 'Light Diseases And Disorders Of Pigmentation', 'Atopic Dermatitis Photos', 'Actinic Keratosis Basal Cell Carcinoma And Other Malignant Lesions', 'Melanoma Skin Cancer Nevi And Moles', 'Cellulitis Impetigo And Other Bacterial Infections', 'Hair Loss Photos Alopecia And Other Hair Diseases', 'Herpes Hpv And Other Stds Photos', 'Fu Nail Fungus', 'Ba  Cellulitis']
Total classes: 20
Class 'Acne And Rosacea Photos' has 6837 images
Class 'Malignant' has 6762 images
Class 'Fu Athlete Foot' has 8054 images
Class 'Eczema Photos' has 6715 images
Class 'Fu Ringworm' has 8129 images
Class 'Benign' has 6459 images
Class 'Exanthems And Drug Eruptions' has 7750 images
Class 'Lupus And Other Connective Tissue Diseases' has 7734 imag

In [4]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

In [5]:
import os

# Suppress TensorFlow Python logs
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Suppress ML Compute / Metal debug info
os.environ["TF_METAL_ENABLE_LOGGING"] = "0"
os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"

import tensorflow as tf
tf.debugging.set_log_device_placement(False)


In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

In [7]:
DATA_ROOT = "dataset"   # <-- change to your dataset folder
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 15
VAL_SPLIT = 0.15
SEED = 1337

In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_ROOT,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_ROOT,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Classes: {num_classes}")

Found 150686 files belonging to 20 classes.
Using 128084 files for training.
Found 150686 files belonging to 20 classes.
Using 22602 files for validation.
Classes: 20


In [9]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.prefetch(buffer_size=AUTOTUNE)

In [10]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
])

In [11]:
base_model = keras.applications.EfficientNetB0(
    include_top=False,
    weights="weights_mobilenet_v3_large_224_1.0_float_no_top_v2.h5",
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling="avg",
)

In [12]:
base_model.trainable = False  # freeze for transfer learning

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = keras.applications.efficientnet.preprocess_input(x)
x = base_model(x, training=False)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)


In [13]:
model = keras.Model(inputs, outputs)

In [14]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)


In [15]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2),
]

In [16]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
)

Epoch 1/15
  98/4003 ━━━━━━━━━━━━━━━━━━━━ 34:50 535ms/step - accuracy: 0.0951 - loss: 3.0044

KeyboardInterrupt: 

In [ ]:
base_model.trainable = True
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

fine_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=callbacks,
)

NameError: name 'base_model' is not defined

In [ ]:
model.save("skin_disease_model_tf.h5")
model.save("skin_disease_model_tf.keras")